# MNIST LSTMConv2D Example
Adapted from [github: TensorFlow Examples](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb).

Uses a custom LSTM cell that implements the LSTMConv2D op like in Keras.


# TODO

- Metric: Round values, percentage of correctly "classified"
- RNNConv2dCell: Initial-state: conv_step ?

In [1]:
from __future__ import print_function

import os
import sys

# add path to libraries for ipython
sys.path.append(os.path.expanduser("~/libs"))

import random

import numpy as np
import tensorflow as tf
import tensortools as tt

In [2]:
BATCH_SIZE = 32
NUM_EXAMPES = 50000

TRAIN_EXAMPLES = 6000
TEST_EXAMPLES = 1000

LEARGNING_RATE = 0.001
DISPLAY_STEP = 10

TIME_STEPS = 10
KERNEL_FILTERS = 1
KERNEL_SIZE = 3
IMAGE_SIZE = 28
CHANNELS = 1
EPOCHS = 10

### Input data

In [3]:
class MovingBlocksInput(object):
    
    def __init__(self, width, height, channels):
        self.train_data_list = []
        self.train_labels_list = []
        self.test_data_list = []
        self.test_labels_list = []
        self.train_counter = 0
        self.test_counter = 0
        self.image_width = width
        self.image_height = height
        self.image_channels = channels


    def _generate_image(self, positions):
        image = np.random.uniform(-1, -0.5, (self.image_height, self.image_width, self.image_channels))
        for p_idx in xrange(len(positions)):
            position = positions[p_idx]
            for x in xrange(position[0]-2, position[0] + 4):
                if x < 0 or x >= self.image_width:
                    continue
                for y in xrange(position[1] - 2, position[1] + 4):
                    if y < 0 or y >= self.image_height:
                        continue
                    image[y, x, 0] = 1.0
        return image
    
    
    def _generate_single_sequence(self, time_steps):
        x_pos1 = random.randint(0, self.image_width)
        y_pos1 = random.randint(0, self.image_height)
        x_step1 = random.randint(-2, 3)
        y_step1 = random.randint(-2, 3)
        x_pos2 = random.randint(0, self.image_width)
        y_pos2 = random.randint(0, self.image_height)
        x_step2 = random.randint(-2, 3)
        y_step2 = random.randint(-2, 3)
        x_pos3 = random.randint(0, self.image_width)
        y_pos3 = random.randint(0, self.image_height)
        x_step3 = random.randint(-2, 3)
        y_step3 = random.randint(-2, 3)
        current_list = []
        for t in xrange(time_steps):
            pos_list = [(x_pos1, y_pos1), (x_pos2, y_pos2), (x_pos3, y_pos3)]
            image = self._generate_image(pos_list)
            current_list.append(image)
            x_pos1 += x_step1
            y_pos1 += y_step1
            x_pos2 += x_step2
            y_pos2 += y_step2
            x_pos3 += x_step3
            y_pos3 += y_step3
        pos_list = [(x_pos1, y_pos1), (x_pos2, y_pos2), (x_pos3, y_pos3)]
        label = self._generate_image(pos_list)
        return np.array(current_list), label
    
    
    def generate_data(self, nb_train, nb_test, time_steps):
        for i in xrange(nb_train):
            seq, label = self._generate_single_sequence(time_steps)
            self.train_data_list.append(seq)
            self.train_labels_list.append(label)
        self.train_data_list = np.asarray(self.train_data_list)
        self.train_labels_list = np.asarray(self.train_labels_list)
        for j in xrange(nb_test):
            seq, label = self._generate_single_sequence(time_steps)
            self.test_data_list.append(seq)
            self.test_labels_list.append(label)
        self.test_data_list = np.asarray(self.test_data_list)
        self.test_labels_list = np.asarray(self.test_labels_list)
 

    def get_train_batch(self, batch_size):
        if self.train_counter + batch_size > len(self.train_data_list):
            self.train_counter = 0
            # shuffle  
            p = np.random.permutation(len(self.train_data_list))
            train_data_list = self.train_data_list[p]
            train_labels_list = self.train_labels_list[p]

        res_data = self.train_data_list[self.train_counter:self.train_counter+batch_size]
        res_labels = self.train_labels_list[self.train_counter:self.train_counter+batch_size]

        self.train_counter += batch_size
        return res_data, res_labels
    
    
    def get_test_batch(self, batch_size):
        self.test_counter = self.test_counter
        if self.test_counter + batch_size > len(self.test_data_list):
            self.test_counter = 0
            # shuffle  
            p = np.random.permutation(len(self.test_data_list))
            self.test_data_list = self.test_data_list[p]
            self.test_labels_list = self.test_labels_list[p]

        res_data = self.test_data_list[self.test_counter:self.test_counter+batch_size]
        res_labels = self.test_labels_list[self.test_counter:self.test_counter+batch_size]

        self.test_counter += batch_size
        return res_data, res_labels

### Graph construction

In [4]:
g = tf.Graph()

In [5]:
def RNN(x):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    print(x.get_shape())
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2, 3, 4])  # (time_steps, batch_size, )
    print(x.get_shape())
    # Reshaping to (n_steps*batch_size, n_input)
    #x = tf.reshape(x, [-1, N_INPUT])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, TIME_STEPS, x)
    print(x[0].get_shape())
    x = [tf.squeeze(i, (0,)) for i in x]
    print(x[0].get_shape())

    # Define a lstm cell with tensorflow
    lstm_cell = tt.recurrent.BasicLSTMConv2DCell(3, 3, 32,
                                                 IMAGE_SIZE, IMAGE_SIZE,
                                                 forget_bias=1.0)

    # Get lstm cell output
    outputs, states = tt.recurrent.rnn_conv2d(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return outputs[-1]

In [6]:
with g.as_default():
    x = tf.placeholder(tf.float32, [None, TIME_STEPS, IMAGE_SIZE, IMAGE_SIZE, CHANNELS], "X")
    y_ = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, CHANNELS], "Y_")

    out = RNN(x)
    
    pred = tt.network.conv2d_transpose("Deconv", out, 1, 3, 3, 1, 1)

(?, 10, 28, 28, 1)
(10, ?, 28, 28, 1)
(1, ?, 28, 28, 1)
(?, 28, 28, 1)
('state-size: ', LSTMStateTuple(c=(28, 28, 32), h=(28, 28, 32)))
('state_size_flat', ((28, 28, 32), (28, 28, 32)))
('structure: ', LSTMStateTuple(c=(28, 28, 32), h=(28, 28, 32)))
('flat structure: ', ((28, 28, 32), (28, 28, 32)))
size: flat: 2, state: 2
('zeros', LSTMStateTuple(c=<tf.Tensor 'RNNConv2D/zeros:0' shape=(?, 28, 28, 32) dtype=float32>, h=<tf.Tensor 'RNNConv2D/zeros_1:0' shape=(?, 28, 28, 32) dtype=float32>))
('state None c: ', <tf.Tensor 'RNNConv2D/zeros:0' shape=(?, 28, 28, 32) dtype=float32>)
('state None h: ', <tf.Tensor 'RNNConv2D/zeros_1:0' shape=(?, 28, 28, 32) dtype=float32>)
('c', <tf.Tensor 'RNNConv2D/zeros:0' shape=(?, 28, 28, 32) dtype=float32>)
('h', <tf.Tensor 'RNNConv2D/zeros_1:0' shape=(?, 28, 28, 32) dtype=float32>)
('i', TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)]))
('j', TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)]))
('f', Ten

In [7]:
with g.as_default():
    with tf.name_scope("Train"):
        cost = tf.reduce_mean(tf.square(pred - y_))
        optimizer = tf.train.AdamOptimizer(learning_rate=LEARGNING_RATE).minimize(cost)

    with tf.name_scope("Accuracy"):
        correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))    

In [8]:
with g.as_default():
    # Launch the graph
    sess = tf.InteractiveSession()
    sess.run(tf.initialize_all_variables())

    tt.visualization.show_graph(sess.graph_def)

    # TODO: remove
    inputs = MovingBlocksInput(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
    inputs.generate_data(TRAIN_EXAMPLES, TEST_EXAMPLES, TIME_STEPS)

    step = 1
    # Keep training until reach max iterations
    while step * BATCH_SIZE < NUM_EXAMPES:
        batch_x, batch_y = inputs.get_train_batch(BATCH_SIZE)
        
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y_: batch_y})
        if step % DISPLAY_STEP == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y_: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y_: batch_y})
            print("@{} - {}/{} examples: Minibatch Loss= {:.6f}, Training Accuracy= {:.5f}".format(
                    step,
                    str(step*BATCH_SIZE),
                    NUM_EXAMPES,
                    loss,
                    acc))

        step += 1
    print("Optimization Finished!")

@10 - 320/50000 examples: Minibatch Loss= 0.173726, Training Accuracy= 0.07254
@20 - 640/50000 examples: Minibatch Loss= 0.116708, Training Accuracy= 0.07254
@30 - 960/50000 examples: Minibatch Loss= 0.096231, Training Accuracy= 0.05580
@40 - 1280/50000 examples: Minibatch Loss= 0.065499, Training Accuracy= 0.04576
@50 - 1600/50000 examples: Minibatch Loss= 0.072474, Training Accuracy= 0.04241
@60 - 1920/50000 examples: Minibatch Loss= 0.066904, Training Accuracy= 0.04799
@70 - 2240/50000 examples: Minibatch Loss= 0.070868, Training Accuracy= 0.05022
@80 - 2560/50000 examples: Minibatch Loss= 0.063123, Training Accuracy= 0.05246
@90 - 2880/50000 examples: Minibatch Loss= 0.047748, Training Accuracy= 0.03906
@100 - 3200/50000 examples: Minibatch Loss= 0.064154, Training Accuracy= 0.05357
@110 - 3520/50000 examples: Minibatch Loss= 0.044613, Training Accuracy= 0.04911
@120 - 3840/50000 examples: Minibatch Loss= 0.049483, Training Accuracy= 0.02567
@130 - 4160/50000 examples: Minibatch Lo

KeyboardInterrupt: 

In [ ]:
with g.as_default():
    batch_x, batch_y = inputs.get_test_batch(1)
    
    # remove batch_dim
    batch_x = np.squeeze(batch_x, axis=0)
    batch_y = np.squeeze(batch_y, axis=0)   
    
    print('IN:')
    for i in range(TIME_STEPS):
        #print_image(test_x[i])
        tt.visualization.display_array(batch_x[i] * 127.5 + 127.5)
    print('TARGET:')
    #print_image(test_y)
    tt.visualization.display_array(batch_y * 127.5 + 127.5)
    print('PREDICTION:')
    batch_x = np.expand_dims(batch_x, axis=0)
    batch_y = np.expand_dims(batch_y, axis=0)
    prediction, acc = sess.run([pred, accuracy], feed_dict={x: batch_x, y_: batch_y})
    prediction_squeezed = np.squeeze(prediction, axis=0)
    #print_image(prediction_squeezed)
    tt.visualization.display_array(prediction_squeezed * 127.5 + 127.5)
    print('Test accuracy', acc)